In [11]:
from google.colab import files
import pandas as pd

uploaded = files.upload()

df = pd.read_csv("creditcard.csv")

# first few rows
df.head()

# Counts columns and rows
print(df.shape)

# Column names
print(df.columns)

# Counts fraud vs non-fraud rows
print((df["Class"] == 0).sum())
print((df["Class"] == 1).sum())



Saving creditcard.csv to creditcard (4).csv
(284807, 31)
Index(['Time', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 'V10',
       'V11', 'V12', 'V13', 'V14', 'V15', 'V16', 'V17', 'V18', 'V19', 'V20',
       'V21', 'V22', 'V23', 'V24', 'V25', 'V26', 'V27', 'V28', 'Amount',
       'Class'],
      dtype='object')
284315
492
